# Image Analysis Tool

## Overview
We want to find clusters in SEM PV images.

## Features

### 🖼️ Analysis Widgets
You have two options. 
1. Edge threshold - select the sensitivity for edge detection
2. Kernel size, select the size of holes in the edges which will be counted as continuous

### 🔍 Plotting
You have two filter options:
1. Min cluster size - rmeove small artefact clusters of the given size
2. Max boarder pixels - remove boarder clusters (chopping off)
---

In [1]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from skimage import io, feature
from scipy import ndimage
from sklearn.cluster import DBSCAN
import io as python_io

# Global variables for uploaded images
uploaded_images = {}
current_image = None
current_banner = None
current_filename = None

print("Image Analysis Tool - Upload images using the widget below")

Image Analysis Tool - Upload images using the widget below


In [2]:
def load_and_display_image(filename_or_data):
    """Load and display an image with banner detection and cropping"""
    try:
        # Handle both filename (string) and uploaded data (bytes)
        if isinstance(filename_or_data, str):
            image = io.imread(filename_or_data)
            display_name = filename_or_data
        else:
            # Handle uploaded file data
            image = io.imread(python_io.BytesIO(filename_or_data))
            display_name = "Uploaded Image"
            
        banner_height = 80
        
        # Crop the main image (remove banner)
        main_image = image[:-banner_height, :] if banner_height > 0 else image
        banner = image[-banner_height:, :] if banner_height > 0 else None
        
        # Create figure with subplots
        if banner is not None:
            fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), 
                                         gridspec_kw={'height_ratios': [4, 1]})
        else:
            fig, ax1 = plt.subplots(1, 1, figsize=(10, 8))
        
        # Display the main image
        if len(main_image.shape) == 2:  # Grayscale
            im1 = ax1.imshow(main_image, cmap='gray')
            plt.colorbar(im1, ax=ax1, label='Intensity')
        elif len(main_image.shape) == 3:  # Color or multi-channel
            if main_image.shape[2] == 3:  # RGB
                ax1.imshow(main_image)
            else:  # Multi-channel, show first channel
                im1 = ax1.imshow(main_image[:,:,0], cmap='gray')
                plt.colorbar(im1, ax=ax1, label='Intensity (Channel 0)')
        
        ax1.set_title(f'Main Image: {display_name}')
        ax1.axis('on')
        
        # Display the banner if detected
        if banner is not None:
            if len(banner.shape) == 2:
                ax2.imshow(banner, cmap='gray', aspect='auto')
            elif len(banner.shape) == 3:
                if banner.shape[2] == 3:
                    ax2.imshow(banner, aspect='auto')
                else:
                    ax2.imshow(banner[:,:,0], cmap='gray', aspect='auto')
            ax2.set_title('Banner/Info Section')
            ax2.axis('on')
        
        plt.tight_layout()
        plt.show()
        return main_image, banner
        
    except Exception as e:
        print(f"Error loading image {display_name}: {str(e)}")
        return None, None

def load_and_display_image_data_only(filename_or_data):
    """Load an image without displaying it"""
    try:
        # Handle both filename (string) and uploaded data (bytes)
        if isinstance(filename_or_data, str):
            image = io.imread(filename_or_data)
            display_name = filename_or_data
        else:
            # Handle uploaded file data
            image = io.imread(python_io.BytesIO(filename_or_data))
            display_name = "Uploaded Image"
            
        banner_height = 80
        main_image = image[:-banner_height, :] if banner_height > 0 else image
        banner = image[-banner_height:, :] if banner_height > 0 else None
        return main_image, banner
    except Exception as e:
        print(f"Error loading image {display_name}: {str(e)}")
        return None, None

In [3]:
# Image analysis utility functions

def show_image_histogram(image, bins=50):
    """Display histogram of image intensities"""
    if image is None:
        print("No image loaded!")
        return
    
    plt.figure(figsize=(10, 4))
    
    if len(image.shape) == 2:  # Grayscale
        plt.subplot(1, 2, 1)
        plt.hist(image.flatten(), bins=bins, alpha=0.7, color='gray')
        plt.title('Intensity Histogram')
        plt.xlabel('Intensity')
        plt.ylabel('Frequency')
        
        plt.subplot(1, 2, 2)
        plt.imshow(image, cmap='gray')
        plt.title('Image')
        plt.axis('off')
        
    elif len(image.shape) == 3:  # Multi-channel
        plt.subplot(1, 2, 1)
        for i in range(min(3, image.shape[2])):
            plt.hist(image[:,:,i].flatten(), bins=bins, alpha=0.5, 
                    label=f'Channel {i}')
        plt.title('Intensity Histogram by Channel')
        plt.xlabel('Intensity')
        plt.ylabel('Frequency')
        plt.legend()
        
        plt.subplot(1, 2, 2)
        if image.shape[2] == 3:
            plt.imshow(image)
        else:
            plt.imshow(image[:,:,0], cmap='gray')
        plt.title('Image')
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

def get_image_stats(image):
    """Print detailed statistics about the image"""
    if image is None:
        print("No image loaded!")
        return
    
    print("=== IMAGE STATISTICS ===")
    print(f"Shape: {image.shape}, Type: {image.dtype}")
    print(f"Min: {image.min()}, Max: {image.max()}, Mean: {image.mean():.2f}")
    
    if len(image.shape) > 2:  # Multi-channel
        for i in range(image.shape[2]):
            channel = image[:,:,i]
            print(f"Channel {i}: Min: {channel.min()}, Max: {channel.max()}, Mean: {channel.mean():.2f}")

def show_main_image_only(image):
    """Display only the main image without banner"""
    if image is None:
        print("No image loaded!")
        return
    
    plt.figure(figsize=(10, 8))
    
    if len(image.shape) == 2:  # Grayscale
        plt.imshow(image, cmap='gray')
        plt.colorbar(label='Intensity')
    elif len(image.shape) == 3:  # Color or multi-channel
        if image.shape[2] == 3:  # RGB
            plt.imshow(image)
        else:  # Multi-channel, show first channel
            plt.imshow(image[:,:,0], cmap='gray')
            plt.colorbar(label='Intensity (Channel 0)')
    
    plt.title('Main Image')
    plt.axis('on')
    plt.tight_layout()
    plt.show()

In [ ]:
# Edge Detection with Interactive Controls

def compute_canny_edges(threshold, current_img, sigma=1):
    """Helper function to compute edges using Canny algorithm"""
    # Convert to grayscale if needed
    if len(current_img.shape) == 3:
        gray_image = np.dot(current_img[...,:3], [0.2989, 0.5870, 0.1140])
    else:
        gray_image = current_img.copy()
    
    gray_image = gray_image.astype(np.float64)
    gray_image = (gray_image - gray_image.min()) / (gray_image.max() - gray_image.min())
    
    # For Canny, use threshold as high threshold, low threshold is half
    edges_raw = feature.canny(gray_image, low_threshold=threshold*0.9, high_threshold=threshold,sigma=sigma)
    return edges_raw.astype(np.uint8), edges_raw.astype(np.uint8), edges_raw.astype(np.float64)

def display_edge_detection_interactive():
    """Create interactive edge detection display with threshold slider"""
    global current_image, edge_output
    
    if current_image is None:
        print("No image loaded!")
        return
    
    # Create threshold slider
    threshold_slider = widgets.BoundedFloatText(
        value=0.3,
        min=0.0,
        max=1.0,
        step=0.01,
        description='Edge Threshold:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='300px')
    )

    sigma_slider = widgets.BoundedFloatText(
        value=3.5,
        min=0.0,
        max=10.0,
        step=0.01,
        description='Sigma:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='300px')
    )
    
    # Create neighborhood analysis button
    analyze_button = widgets.Button(
        description='Analyze Neighborhoods',
        button_style='info',
        tooltip='Analyze neighborhood connectivity',
        layout=widgets.Layout(width='200px')
    )
    
    kernel_size_widget = widgets.BoundedIntText(
        value=18,
        min=1,
        description='Kernel Size:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='150px'),
        tooltip='Size of the kernel for neighborhood analysis'
    )
    
    analysis_output = widgets.Output()
    
    def on_analyze_button_click(button):
        """Handle neighborhood analysis button click"""
        with analysis_output:
            analysis_output.clear_output(wait=True)
            
            current_img = globals().get('current_image', None)
            if current_img is None:
                print("No image loaded!")
                return
            
            threshold = threshold_slider.value
            sigma = sigma_slider.value
            kernel_size = kernel_size_widget.value
            
            # Get edges for analysis
            _, edges, _ = compute_canny_edges(threshold, current_img, sigma)
            plot_neighborhood_analysis_interactive(edges, kernel_size)
    
    analyze_button.on_click(on_analyze_button_click)
    
    def update_edge_display(change=None):
        """Update edge detection display when slider changes"""
        with edge_output:
            clear_output()
            
            current_img = globals().get('current_image', None)
            if current_img is None:
                print("No image loaded! Please upload and select an image first.")
                return
            
            threshold = threshold_slider.value
            sigma = sigma_slider.value

            _, edges, gradient_to_show = compute_canny_edges(threshold, current_img, sigma)
            
            # Create figure with subplots
            fig, axes = plt.subplots(1, 3, figsize=(18, 6))
            
            # Original image
            if len(current_img.shape) == 2:
                axes[0].imshow(current_img, cmap='gray')
            else:
                if current_img.shape[2] == 3:
                    axes[0].imshow(current_img)
                else:
                    axes[0].imshow(current_img[:,:,0], cmap='gray')
            axes[0].set_title('Original Image')
            axes[0].axis('on')
            
            # Edge detection result
            axes[1].imshow(edges, cmap='gray')
            axes[1].set_title(f'Detected Edges (Threshold: {threshold:.3f})')
            axes[1].axis('on')
            
            # Overlay edges on original image
            if len(current_img.shape) == 2:
                overlay_base = np.stack([current_img]*3, axis=-1)
                overlay_base = (overlay_base - overlay_base.min()) / (overlay_base.max() - overlay_base.min())
            else:
                overlay_base = current_img.copy().astype(np.float64)
                if overlay_base.shape[2] > 3:
                    overlay_base = overlay_base[:,:,:3]
                overlay_base = (overlay_base - overlay_base.min()) / (overlay_base.max() - overlay_base.min())
            
            overlay = overlay_base.copy()
            if len(overlay.shape) == 3:
                overlay[edges > 0, 0] = 1.0  # Red channel for edges
            
            axes[2].imshow(overlay)
            axes[2].set_title('Edges Overlaid on Original Image')
            axes[2].axis('on')
            
            plt.tight_layout()
            plt.show()
            
            # Print statistics
            edge_pixels = np.sum(edges > 0)
            total_pixels = edges.size
            edge_percentage = (edge_pixels / total_pixels) * 100
            print(f"Edge pixels: {edge_pixels:,} ({edge_percentage:.2f}% of image)")
    
    # Connect slider to update function
    threshold_slider.observe(update_edge_display, names='value')
    sigma_slider.observe(update_edge_display, names='value')
    
    # Display controls
    controls_row1 = widgets.HBox([threshold_slider, sigma_slider])
    controls_row2 = widgets.HBox([analyze_button, kernel_size_widget])
    controls = widgets.VBox([controls_row1, controls_row2])
    
    display(controls)
    display(edge_output)
    display(analysis_output)
    
    # Initial display
    update_edge_display()
    
    return threshold_slider, analyze_button

edge_output = widgets.Output()

# Image upload and selection setup
def on_image_upload(change):
    """Handle image upload"""
    global current_image, current_banner, current_filename, uploaded_images
    
    if not upload_widget.value:
        return
    
    # Clear previous outputs
    with upload_output:
        upload_output.clear_output()
        
        # Handle the upload widget value format
        # upload_widget.value is a tuple of FileUpload objects
        uploaded_files = upload_widget.value
        print(f"Uploaded {len(uploaded_files)} file(s)")
        
        # Store uploaded images
        for file_upload in uploaded_files:
            filename = file_upload.name
            file_content = file_upload.content
            uploaded_images[filename] = file_content
            print(f"- {filename} ({len(file_content):,} bytes)")
        
        # Update dropdown with uploaded files
        image_dropdown.options = list(uploaded_images.keys())
        if uploaded_images:
            # Set the first image as selected, but don't trigger the handler manually
            # The dropdown observer will handle it automatically
            image_dropdown.value = list(uploaded_images.keys())[0]

def on_image_selection_change(change):
    """Update edge detection when image selection changes"""
    global current_image, current_banner, current_filename, edge_output

    selected_filename = change.get('new', change) if isinstance(change, dict) else change
    
    if selected_filename and selected_filename in uploaded_images:
        current_filename = selected_filename
        current_image, current_banner = load_and_display_image_data_only(uploaded_images[selected_filename])
        if current_image is not None:
            print(f"Loaded: {selected_filename}")
            # Clear the edge output area first
            with edge_output:
                edge_output.clear_output()
            # Display edge detection widgets outside of the output context
            display_edge_detection_interactive()
        else:
            print(f"Failed to load: {selected_filename}")

# Create upload widget
upload_widget = widgets.FileUpload(
    accept='.tif,.tiff,.png,.jpg,.jpeg',  # Accept common image formats
    multiple=True,
    description='Upload Images:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Create dropdown for selecting from uploaded images
image_dropdown = widgets.Dropdown(
    options=[],
    description='Select Image:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px'),
    disabled=False
)

# Create output areas
upload_output = widgets.Output()

# Connect event handlers
upload_widget.observe(on_image_upload, names='value')
image_dropdown.observe(on_image_selection_change, names='value')

# Display upload interface
display(upload_widget)
display(image_dropdown)

display(upload_output)
plot_output = widgets.Output()
display(plot_output)




FileUpload(value=(), accept='.tif,.tiff,.png,.jpg,.jpeg', description='Upload Images:', layout=Layout(width='4…

Dropdown(description='Select Image:', layout=Layout(width='400px'), options=(), style=DescriptionStyle(descrip…

Output()

Output()

In [5]:
# Neighborhood Connectivity Analysis

def create_circular_kernel(size):
    """Create a circular kernel of given size"""
    center = size // 2
    y, x = np.ogrid[:size, :size]
    dist_from_center = np.sqrt((x - center)**2 + (y - center)**2)
    radius = center
    circular_mask = dist_from_center <= radius
    kernel = circular_mask.astype(np.float32)
    kernel[center, center] = 0  # Don't count center pixel
    return kernel

def cluster(data, epsilon, N):
    """DBSCAN clustering with euclidean distance"""
    db = DBSCAN(eps=epsilon, min_samples=N).fit(data)
    labels = db.labels_
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    clusters = [data[labels == i] for i in range(n_clusters)]
    return clusters, n_clusters

def analyze_neighborhood_connectivity(edges, kernel_size):
    """Analyze neighborhood connectivity using circular kernels"""
    from scipy.ndimage import convolve
    
    print(f"  - Creating circular kernel (size: {kernel_size})...")
    # Create circular kernel
    kernel = create_circular_kernel(kernel_size)
    
    print(f"  - Computing neighbor counts for {edges.shape[0]}x{edges.shape[1]} image...")
    # Count non-zero neighbors for each pixel
    neighbor_count = convolve(edges.astype(np.float32), kernel, mode='constant', cval=0)
    
    # Find isolated pixels (neighbor_count == 0)
    pixels_with_zero_neighbors = (neighbor_count == 0)
    isolated_positions = np.where(pixels_with_zero_neighbors)
    kernel_centers = list(zip(isolated_positions[0], isolated_positions[1]))
    
    print(f"  - Found {len(kernel_centers)} isolated pixel centers")
    
    if not kernel_centers:
        print("  - No isolated centers found, returning empty list")
        return []
    
    print(f"  - Creating initial regions from {len(kernel_centers)} centers...")
    # Create initial regions
    regions = [{'centers': [(y, x)]} for y, x in kernel_centers]
    centers = np.array([region['centers'][0] for region in regions], dtype=np.int32)
    
    print("  - Clustering nearby centers using DBSCAN...")
    # Cluster nearby centers
    connected_components, n_regions = cluster(centers, 3, 10)
    
    print(f"  - Found {n_regions} connected components, merging regions...")
    # Merge regions within each connected component
    all_kernel_centers = []
    for i, component in enumerate(connected_components):        
        merged_centers = []
        merged_pixels = set()
        
        for center in component:
            y, x = center
            merged_centers.append(center)
            
            # Get all pixels within the circular kernel for this center
            kernel = create_circular_kernel(kernel_size)
            half_size = kernel_size // 2
            
            # Calculate bounds for the kernel region
            y_start = max(0, y - half_size)
            y_end = min(edges.shape[0], y + half_size + 1)
            x_start = max(0, x - half_size)
            x_end = min(edges.shape[1], x + half_size + 1)
            
            # Get kernel section that fits within image bounds
            ky_start = max(0, half_size - y)
            ky_end = ky_start + (y_end - y_start)
            kx_start = max(0, half_size - x)
            kx_end = kx_start + (x_end - x_start)
            kernel_section = kernel[ky_start:ky_end, kx_start:kx_end]
            
            # Find all pixels within the circular kernel region
            kernel_pixels = []
            for ky, kx in np.argwhere(kernel_section > 0):
                pixel_y = y_start + ky
                pixel_x = x_start + kx
                kernel_pixels.append((pixel_y, pixel_x))
            
            kernel_pixels.append((y, x))  # Add center pixel
            merged_pixels.update(kernel_pixels)
        
        # Create merged region
        merged_region = {
            'centers': merged_centers,
            'pixels': list(merged_pixels),
            'kernel_size': kernel_size
        }
        all_kernel_centers.append(merged_region)
    
    print(f"  - Analysis complete! Created {len(all_kernel_centers)} merged regions")
    return all_kernel_centers

def count_border_pixels(region, image_shape):
    """Count how many pixels in a region are at the image border"""
    border_count = 0
    height, width = image_shape[:2]
    
    for pixel_y, pixel_x in region['pixels']:
        if (pixel_y == 0 or pixel_y == height - 1 or 
            pixel_x == 0 or pixel_x == width - 1):
            border_count += 1
    
    return border_count

def filter_clusters(clusters, min_cluster_size, max_border_pixels, image_shape):
    """Filter clusters based on size and border pixel criteria"""
    filtered_clusters = []
    
    for cluster in clusters:
        cluster_size = len(cluster['pixels'])
        border_pixels = count_border_pixels(cluster, image_shape)
        
        if cluster_size >= min_cluster_size and border_pixels <= max_border_pixels:
            cluster['border_pixels'] = border_pixels
            filtered_clusters.append(cluster)
    
    return filtered_clusters

def create_analysis_plots(clusters, current_img, kernel_size, min_cluster_size, max_border_pixels, total_clusters):
    """Create the three analysis plots with filtered cluster data"""
    fig = plt.figure(figsize=(20, 18))
    ax3 = plt.subplot(3, 1, 1)  # Top: overlay image
    ax1 = plt.subplot(3, 1, 2)  # Middle: log histogram
    ax2 = plt.subplot(3, 1, 3)  # Bottom: linear histogram
    
    # Plot histograms if clusters exist
    if clusters:
        pixel_counts = [len(region['pixels']) for region in clusters]
        
        if len(pixel_counts) > 0:
            num_buckets = 20
            min_size = min(pixel_counts)
            max_size = max(pixel_counts)
            
            # Log-scaled histogram
            log_min = np.log10(min_size + 1)
            log_max = np.log10(max_size + 1)
            log_edges = np.linspace(log_min, log_max, num_buckets+1)
            bucket_edges = 10**log_edges - 1
            bucket_edges[0] = min_size
            bucket_edges[-1] = max_size + 1
            
            bucket_counts = np.zeros(num_buckets)
            for size in pixel_counts:
                bucket_idx = np.searchsorted(bucket_edges[1:], size)
                bucket_idx = min(bucket_idx, num_buckets - 1)
                bucket_counts[bucket_idx] += 1
            
            bars = ax1.bar(range(num_buckets), bucket_counts, alpha=0.7, color='steelblue', edgecolor='black')
            ax1.set_xlabel('Region Size Buckets (Log Scale)')
            ax1.set_ylabel('Number of Regions')
            ax1.set_title(f'Distribution of Region Sizes ({len(clusters)} filtered regions, Log-Scaled)')
            ax1.grid(True, alpha=0.3, axis='y')
            
            # Set x-axis labels for log histogram
            bucket_labels = []
            for i in range(num_buckets):
                start = int(bucket_edges[i])
                end = int(bucket_edges[i+1]) - 1 if i < num_buckets - 1 else int(bucket_edges[i+1])
                bucket_labels.append(f'{start}-{end}')
            
            # Show every 2nd or 3rd label to avoid overcrowding
            label_step = max(1, num_buckets // 10)  # Show about 10 labels max
            tick_positions = range(0, num_buckets, label_step)
            tick_labels = [bucket_labels[i] for i in tick_positions]
            
            ax1.set_xticks(tick_positions)
            ax1.set_xticklabels(tick_labels, rotation=45, ha='right')
            
            # Add value labels on bars
            for i, (bar, count) in enumerate(zip(bars, bucket_counts)):
                if count > 0:
                    height = bar.get_height()
                    ax1.annotate(f'{int(count)}', xy=(bar.get_x() + bar.get_width()/2, height),
                                xytext=(0, 3), textcoords='offset points',
                                ha='center', va='bottom', fontsize=10)
            
            # Linear histogram
            lin_bucket_edges = np.linspace(min_size, max_size + 1, num_buckets + 1)
            lin_bucket_counts = np.zeros(num_buckets)
            for size in pixel_counts:
                bucket_idx = np.searchsorted(lin_bucket_edges[1:], size)
                bucket_idx = min(bucket_idx, num_buckets - 1)
                lin_bucket_counts[bucket_idx] += 1
            
            lin_bars = ax2.bar(range(num_buckets), lin_bucket_counts, alpha=0.7, color='darkgreen', edgecolor='black')
            ax2.set_xlabel('Region Size Buckets (Linear Scale)')
            ax2.set_ylabel('Number of Regions')
            ax2.set_title(f'Distribution of Region Sizes ({len(clusters)} filtered regions, Linear)')
            ax2.grid(True, alpha=0.3, axis='y')
            
            # Set x-axis labels for linear histogram
            lin_bucket_labels = []
            for i in range(num_buckets):
                start = int(lin_bucket_edges[i])
                end = int(lin_bucket_edges[i+1]) - 1 if i < num_buckets - 1 else int(lin_bucket_edges[i+1])
                lin_bucket_labels.append(f'{start}-{end}')
            
            # Show labels with appropriate step size
            lin_label_step = max(1, num_buckets // 10)  # Show about 10 labels max
            lin_tick_positions = range(0, num_buckets, lin_label_step)
            lin_tick_labels = [lin_bucket_labels[i] for i in lin_tick_positions]
            
            ax2.set_xticks(lin_tick_positions)
            ax2.set_xticklabels(lin_tick_labels, rotation=45, ha='right')
            
            # Add value labels on linear bars
            for i, (bar, count) in enumerate(zip(lin_bars, lin_bucket_counts)):
                if count > 0:
                    height = bar.get_height()
                    ax2.annotate(f'{int(count)}', xy=(bar.get_x() + bar.get_width()/2, height),
                                xytext=(0, 3), textcoords='offset points',
                                ha='center', va='bottom', fontsize=10)
            
            # Add filter info
            filter_text = f'Filtered: {len(clusters)} of {total_clusters} clusters (Size≥{min_cluster_size}, Border≤{max_border_pixels})'
            stats_text = f'Min: {min_size} px, Max: {max_size} px, Mean: {np.mean(pixel_counts):.1f} px'
            ax2.text(0.02, 0.98, f'{stats_text}\n{filter_text}', transform=ax2.transAxes, 
                    verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        else:
            ax1.text(0.5, 0.5, 'No regions pass filter criteria', ha='center', va='center', 
                    transform=ax1.transAxes, fontsize=14)
            ax2.text(0.5, 0.5, 'No regions pass filter criteria', ha='center', va='center', 
                    transform=ax2.transAxes, fontsize=14)
    else:
        ax1.text(0.5, 0.5, 'No regions found', ha='center', va='center', 
                transform=ax1.transAxes, fontsize=14)
        ax2.text(0.5, 0.5, 'No regions found', ha='center', va='center', 
                transform=ax2.transAxes, fontsize=14)
    
    # Plot overlay image
    if current_img is not None:
        # Prepare base image for overlay
        if len(current_img.shape) == 2:  # Grayscale
            overlay_base = np.stack([current_img]*3, axis=-1)
            overlay_base = (overlay_base - overlay_base.min()) / (overlay_base.max() - overlay_base.min())
        else:  # Color or multi-channel
            overlay_base = current_img.copy().astype(np.float64)
            if overlay_base.shape[2] > 3:
                overlay_base = overlay_base[:,:,:3]
            overlay_base = (overlay_base - overlay_base.min()) / (overlay_base.max() - overlay_base.min())
        
        overlay_image = overlay_base.copy()
        
        if clusters:
            region_colors = plt.cm.Set3(np.linspace(0, 1, len(clusters)))
            np.random.shuffle(region_colors)
            
            for i, region_info in enumerate(clusters):
                region_color = region_colors[i % len(region_colors)]
                
                # Highlight pixels in the region
                for pixel_y, pixel_x in region_info['pixels']:
                    if 0 <= pixel_y < overlay_image.shape[0] and 0 <= pixel_x < overlay_image.shape[1]:
                        overlay_image[pixel_y, pixel_x] = overlay_image[pixel_y, pixel_x] * 0.8 + np.array(region_color[:3]) * 0.2
                
                # Add region label
                if region_info['centers']:
                    centers_array = np.array(region_info['centers'])
                    mean_y = np.mean(centers_array[:, 0])
                    mean_x = np.mean(centers_array[:, 1])
                    
                    ax3.annotate(f"{i+1:02d}", (mean_x, mean_y), 
                               fontsize=5, fontweight='bold', 
                               color='white', ha='center', va='center',
                               bbox=dict(boxstyle='round,pad=0.3', 
                                       facecolor='black', alpha=0.7, edgecolor='none'))
        
        ax3.imshow(overlay_image)
        title_text = f'Filtered Regions Overlaid on Original Image\n'
        title_text += f'Showing: {len(clusters)} of {total_clusters} regions '
        title_text += f'(Size≥{min_cluster_size}, Border≤{max_border_pixels})'
        ax3.set_title(title_text)
        ax3.axis('on')
    else:
        ax3.text(0.5, 0.5, 'No image available for overlay', 
                ha='center', va='center', transform=ax3.transAxes, fontsize=14)
        ax3.set_title('Image Overlay (No Image Loaded)')
    
    plt.tight_layout()
    plt.show()

def plot_neighborhood_analysis_interactive(edges, kernel_size):
    """Interactive plot of neighborhood connectivity analysis with filtering widgets"""
    print(f"Starting neighborhood analysis with kernel size {kernel_size}...")
    print("Step 1: Analyzing neighborhood connectivity...")
    
    # Get all clusters first
    progress_output = widgets.Output()
    display(progress_output)
    with progress_output:
        progress_output.clear_output(wait=True)
    all_clusters = analyze_neighborhood_connectivity(edges, kernel_size)
    
    print(f"Step 2: Found {len(all_clusters)} initial clusters")
    
    # Set up default filter values
    default_min_size = (kernel_size * kernel_size) // 2
    default_max_border = 10
    
    print("Step 3: Setting up interactive widgets...")
    
    # Create filter widgets
    min_size_widget = widgets.BoundedFloatText(
        value=default_min_size,
        min=1,
        max=10000,
        step=1,
        description='Min Cluster Size:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='200px'),
        tooltip='Minimum number of pixels in cluster to include'
    )
    
    max_border_widget = widgets.BoundedFloatText(
        value=default_max_border,
        min=0,
        max=1000,
        step=1,
        description='Max Border Pixels:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='200px'),
        tooltip='Maximum number of border pixels allowed in cluster'
    )
    
    def update_plots(change=None):
        """Update plots when filter values change"""
        with progress_output:
            progress_output.clear_output(wait=True)
            
            min_cluster_size = int(min_size_widget.value)
            max_border_pixels = int(max_border_widget.value)
            
            print(f"Filtering clusters: min_size={min_cluster_size}, max_border={max_border_pixels}")
            
            current_img = globals().get('current_image', None)
            if current_img is None:
                print("No image available!")
                return
            
            # Filter clusters and create plots
            print("Applying filters to clusters...")
            filtered_clusters = filter_clusters(all_clusters, min_cluster_size, max_border_pixels, current_img.shape)
            progress_output.clear_output()
            print(f"Filtered down to {len(filtered_clusters)} clusters")

        with plot_output:
            plot_output.clear_output(wait=True)
            print("Generating plots...")
            create_analysis_plots(filtered_clusters, current_img, kernel_size, 
                                min_cluster_size, max_border_pixels, len(all_clusters))
            print("Analysis complete!")
    
    # Connect widgets to update function
    min_size_widget.observe(update_plots, names='value')
    max_border_widget.observe(update_plots, names='value')
    
    # Display controls and initial plot
    controls = widgets.HBox([min_size_widget, max_border_widget])
    display(controls)
    
    print("Step 4: Creating initial visualization...")
    update_plots()

